In [1905]:
import tweepy
import pandas as pd
client = tweepy.Client("AAAAAAAAAAAAAAAAAAAAAAlMbQEAAAAAF9SMLT8PB%2BroSf8KnkkEu7njA8M%3DFUBoy059uIfXnjFy0fGY8GOwT5C91aEC5krMpFoAcRYMC9NPzn")
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime
import re
import flair
import yfinance as yf
import nltk
from textblob import TextBlob as tb
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\peeta\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [1837]:
def basic_count(query):
    count_df=pd.DataFrame(columns=['time', 'count'])
    response = client.get_recent_tweets_count(query, granularity="hour")
    for count in response.data:
        data={'time':count['end'], 'count':count['tweet_count']}
        count_df=count_df.append(data, ignore_index=True)
    return count_df

In [1770]:
basic_count('putin')

,time,count
0,2022-04-03T02:00:00.000Z,19492
1,2022-04-03T03:00:00.000Z,24312
2,2022-04-03T04:00:00.000Z,22864
3,2022-04-03T05:00:00.000Z,20809
4,2022-04-03T06:00:00.000Z,22085
...,...,...
164,2022-04-09T22:00:00.000Z,22605
165,2022-04-09T23:00:00.000Z,21246
166,2022-04-10T00:00:00.000Z,20308
167,2022-04-10T01:00:00.000Z,16571


In [146]:
query='tesla OR tsla'

In [147]:
pos_query='#bitcoin (Moon OR rocket OR bullish OR up OR positive OR gain) -bearish -lose -crash -down -negative'

In [2275]:
btc_count=basic_count('BTC OR bitcoin').set_index('time')

In [2292]:
eth_count=basic_count('ETH OR ethereum').set_index('time')

In [2293]:
doge_count=basic_count('DOGE OR dogecoin').set_index('time')

In [2299]:
ada_count=basic_count('cardano').set_index('time')

In [2678]:
gme_count=basic_count('gme OR gamestop').set_index('time')

In [2679]:
amc_count=basic_count('AMC').set_index('time')

In [2680]:
tsla_count=basic_count('tsla OR tesla').set_index('time')

In [2681]:
memestock_counts=pd.concat([gme_count,amc_count,tsla_count], join='inner', axis=1)

In [2682]:
memestock_counts.columns=['gme','amc','tsla']

In [2300]:
ada_count

,count
time,
2022-04-03T18:00:00.000Z,1167
2022-04-03T19:00:00.000Z,1483
2022-04-03T20:00:00.000Z,1361
2022-04-03T21:00:00.000Z,1242
2022-04-03T22:00:00.000Z,1399
...,...
2022-04-10T14:00:00.000Z,1213
2022-04-10T15:00:00.000Z,1488
2022-04-10T16:00:00.000Z,1348


In [2301]:
all_counts=pd.concat([btc_count,eth_count,doge_count,ada_count], join='inner', axis=1)

In [2302]:
all_counts.columns=['btc','eth','doge','ada']

In [2303]:
all_counts

,btc,eth,doge,ada
time,,,,
2022-04-03T18:00:00.000Z,22612,33996,4764,1167
2022-04-03T19:00:00.000Z,27461,41185,7089,1483
2022-04-03T20:00:00.000Z,26167,41047,7004,1361
2022-04-03T21:00:00.000Z,22904,41780,6508,1242
2022-04-03T22:00:00.000Z,22145,37787,4876,1399
...,...,...,...,...
2022-04-10T13:00:00.000Z,32089,54028,6222,1409
2022-04-10T14:00:00.000Z,32393,57903,7476,1213
2022-04-10T15:00:00.000Z,32933,61072,11175,1488


In [1841]:
sentiment_model = flair.models.TextClassifier.load('en-sentiment')
def tweets(start_time, end_time, iterations, query):
    datelist = pd.date_range(start_time, end_time, periods=iterations).tolist()
    tweets=[]
    date=[]
    prob=[]
    sentim=[]
    a=0
    for time in datelist:
        if a>1:
            #Fetching Tweets
            response = client.search_recent_tweets(query, max_results=100, start_time=start, end_time=time, tweet_fields=["created_at"])
            response.data
            if type(response.data) != type(None):
                #Data Cleaning
                for i in response.data:
                    tw=i
                    tw=str(i)
                    tw=tw.lower()
                    tw=re.sub(r"http\S+", '',tw)
                    tw=re.sub(r"www.\S+",'',tw)
                    tw=re.sub("[^A-Za-z0-9]"," ",tw)
                    tw=tw.split()
                    tw=' '.join(word for word in tw)
                    #Sentiment analysis
                    if len(tw) > 0:
                        sentence = flair.data.Sentence(tw)
                        sentiment_model.predict(sentence)
                        probability=sentence.labels[0].score
                        sentiment=sentence.labels[0].value
                        tweets.append(tw)
                        date.append(i.created_at)
                        prob.append(probability)
                        sentim.append(sentiment)
        start=time
        a=a+1
        
    tweet_df=pd.DataFrame({'time':date, 'tweet':tweets, 'probability':prob, 'sentiment':sentim})
    tweet_df=tweet_df.sort_values(by='time', ascending=True).set_index('time')
    return tweet_df

2022-04-10 18:44:20,111 loading file C:\Users\peeta\.flair\models\sentiment-en-mix-distillbert_4.pt


In [625]:
BTC=tweets('2022-04-01', '2022-04-07', 100, 'btc OR bitcoin OR crypto -is:retweet lang:en')

In [1774]:
BTC.to_csv('btc.csv')

In [1794]:
def nltk_polar(test):
    tb_polarity = [tb(i).sentiment[0] for i in test['tweet']]
    nltk_polarity = [sid.polarity_scores(i)['compound'] for i in test['tweet']]
    test['tb_polarity']=tb_polarity
    test['nltk_polarity']=nltk_polarity
    return test

In [1821]:
def flair_format(flairtest):
    flairtest=flairtest.drop(columns=['tb_polarity','nltk_polarity'])
    flairtest=flairtest.pivot_table('probability',['time','tweet'], 'sentiment')
    flairtest=flairtest.groupby('time').count()
    flairtest=flairtest.reset_index()
    flairtest['time']=flairtest['time'].round('60min')
    flairtest=flairtest.groupby('time').sum()
    flairtest=flairtest.drop([flairtest.index[0], flairtest.index[-1]])
    flairtest['pct']=(flairtest['POSITIVE']/(flairtest['POSITIVE']+flairtest['NEGATIVE']))
    return flairtest

In [1824]:
def nltk_format(nltktest):
    nltktest=nltktest.drop(columns=['tb_polarity', 'probability', 'sentiment'])
    nltktest=nltktest.reset_index()
    nltktest['time']=nltktest['time'].round('60min')
    nltktest=nltktest.groupby('time').mean()
    return nltktest

In [1797]:
def normil(df):
    df = (df-df.min())/(df.max()-df.min())
    return df

In [2524]:
BTC

,NEGATIVE,POSITIVE,pct,nltk_polarity
time,,,,
2022-04-04 02:00:00+00:00,85,114,0.572864,0.218117
2022-04-04 03:00:00+00:00,79,117,0.596939,0.248215
2022-04-04 04:00:00+00:00,36,64,0.640000,0.233663
2022-04-04 05:00:00+00:00,91,103,0.530928,0.211483
2022-04-04 06:00:00+00:00,75,124,0.623116,0.257756
...,...,...,...,...
2022-04-09 19:00:00+00:00,80,119,0.597990,0.228531
2022-04-09 20:00:00+00:00,48,52,0.520000,0.208677
2022-04-09 21:00:00+00:00,75,122,0.619289,0.228769


In [1876]:
def complete_data(Name, Query):
    Stock=tweets('2022-04-04', '2022-04-10', 250, Query)
    Stock=nltk_polar(Stock)
    Stock_flair=flair_format(Stock)
    Stock_nltk=nltk_format(Stock)
    Stock_comb=Stock_flair.join(Stock_nltk)
    Stock_comb.to_csv(Name)
    return Stock_comb, Stock

In [1946]:
btc = yf.Ticker("BTC-USD")
btc_stock = btc.history(
    start='2022-04-04',
    end='2022-04-11',
    interval='60m'
)
btc_price_normil=normil(btc_stock)

In [1947]:
eth = yf.Ticker("ETH-USD")
eth_stock = eth.history(
    start='2022-04-04',
    end='2022-04-11',
    interval='60m'
)
eth_price_normil=normil(eth_stock)

In [1948]:
doge = yf.Ticker("doge-USD")
doge_stock = doge.history(
    start='2022-04-04',
    end='2022-04-11',
    interval='60m'
)
doge_price_normil=normil(doge_stock)

In [1949]:
ada = yf.Ticker("ADA-USD")
ada_stock = ada.history(
    start='2022-04-04',
    end='2022-04-11',
    interval='60m'
)
ada_price_normil=normil(ada_stock)

## CRYPTO

In [1845]:
ETH = complete_data('ETH', 'eth OR ethereum -is:retweet lang:en')
BTC = complete_data('BTC', 'btc OR bitcoin -is:retweet lang:en')
DOGE = complete_data('DOGE', 'doge OR dogecoin -is:retweet lang:en')

In [1844]:
ADA = complete_data('ADA', 'cardano OR ADA OR Charles Hoskinson -is:retweet lang:en')

In [1851]:
ADA.to_csv('ADA.csv')
BTC.to_csv('BTC.csv')
DOGE.to_csv('DOGE.csv')
ETH.to_csv('ETH.csv')

In [2168]:
ETH_normil=normil(ETH)
BTC_normil=normil(BTC)
DOGE_normil=normil(DOGE)
ADA_normil=normil(ADA)

In [2169]:
ETH_joined=pd.concat([ETH_normil,eth_price_normil], axis=1, join='inner')
BTC_joined=pd.concat([BTC_normil,btc_price_normil], axis=1, join='inner')
DOGE_joined=pd.concat([DOGE_normil,doge_price_normil], axis=1, join='inner')
ADA_joined=pd.concat([ADA_normil,ada_price_normil], axis=1, join='inner')

In [2457]:
ETH_joined_norm=normil(ETH_joined)

In [2170]:
ETH_rolling=ETH_joined.rolling(12).mean()
BTC_rolling=BTC_joined.rolling(12).mean()
DOGE_rolling=DOGE_joined.rolling(12).mean()
ADA_rolling=ADA_joined.rolling(12).mean()

In [2171]:
BTC_rolling_normil=normil(BTC_rolling)
ETH_rolling_normil=normil(ETH_rolling)
DOGE_rolling_normil=normil(DOGE_rolling)
ADA_rolling_normil=normil(ADA_rolling)

In [2176]:
BTC_rolling_normil['shifted_positive'] = BTC_rolling_normil['POSITIVE'].shift(periods=-6)
BTC_rolling_normil['shifted_pct'] = BTC_rolling_normil['pct'].shift(periods=-6)
ETH_rolling_normil['shifted_positive'] = ETH_rolling_normil['POSITIVE'].shift(periods=-6)
ETH_rolling_normil['shifted_pct'] = ETH_rolling_normil['pct'].shift(periods=-6)
DOGE_rolling_normil['shifted_positive'] = DOGE_rolling_normil['POSITIVE'].shift(periods=-6)
DOGE_rolling_normil['shifted_pct'] = DOGE_rolling_normil['pct'].shift(periods=-6)
ADA_rolling_normil['shifted_positive'] = ADA_rolling_normil['POSITIVE'].shift(periods=-6)
ADA_rolling_normil['shifted_pct'] = ADA_rolling_normil['pct'].shift(periods=-6)

In [1846]:
ETH

,NEGATIVE,POSITIVE,pct,nltk_polarity
time,,,,
2022-04-04 02:00:00+00:00,71,120,0.628272,0.318266
2022-04-04 03:00:00+00:00,92,100,0.520833,0.329018
2022-04-04 04:00:00+00:00,50,48,0.489796,0.304906
2022-04-04 05:00:00+00:00,67,124,0.649215,0.343403
2022-04-04 06:00:00+00:00,83,112,0.574359,0.282835
...,...,...,...,...
2022-04-09 19:00:00+00:00,107,87,0.448454,0.231989
2022-04-09 20:00:00+00:00,11,24,0.685714,0.465228
2022-04-09 21:00:00+00:00,94,105,0.527638,0.315555


In [2732]:
all_counts_plt=all_counts.hvplot(xlabel='date', xticks=0, title='Tweet Volume of Different Assets')

In [2716]:
memestock_counts

,gme,amc,tsla
time,,,
2022-04-03T22:00:00.000Z,644,1142,3173
2022-04-03T23:00:00.000Z,2932,1024,3104
2022-04-04T00:00:00.000Z,1249,1194,2761
2022-04-04T01:00:00.000Z,1047,1237,2796
2022-04-04T02:00:00.000Z,901,1684,2887
...,...,...,...
2022-04-10T17:00:00.000Z,627,1232,3544
2022-04-10T18:00:00.000Z,616,1176,4218
2022-04-10T19:00:00.000Z,658,1091,4157


In [2717]:
memestock_all_counts_plt=memestock_counts.reset_index().hvplot(x='time', y=['gme','amc','tsla'], xlabel='date', xticks=0, title='Tweet Volume of Different Meme Stocks')

In [2718]:
memestock_all_counts_plt

:NdOverlay   [Variable]
   :Curve   [time]   (value)

In [1850]:
ETH.hvplot(x='time', y='nltk_polarity')*BTC.hvplot(x='time', y='nltk_polarity')*ADA.hvplot(x='time', y='nltk_polarity')*DOGE.hvplot(x='time', y='nltk_polarity')

:Overlay
   .Curve.I   :Curve   [time]   (nltk_polarity)
   .Curve.II  :Curve   [time]   (nltk_polarity)
   .Curve.III :Curve   [time]   (nltk_polarity)
   .Curve.IV  :Curve   [time]   (nltk_polarity)

In [2419]:
Crypto_Pct=ETH.hvplot(x='time',y='pct', label='ETH', ylabel='Percentage of Tweets Positive', title='Percentage of Tweets that are Positive')*BTC.hvplot(x='time', y='pct', label='BTC')*ADA.hvplot(x='time', y='pct', label='ADA')*DOGE.hvplot(x='time', y='pct', label='DOGE')

In [2382]:
Crypto_Price=ada_price_normil.hvplot(y='Close', ylabel='Normalised Close Price', xlabel='Date', title='Price of Popular Cryptos', label='ADA')*btc_price_normil.hvplot(y='Close', label='BTC')*eth_price_normil.hvplot(y='Close', label='ETH')*doge_price_normil.hvplot(y='Close', label='DOGE')

In [2136]:
ETH_rolling_normil.hvplot(y='nltk_polarity')*BTC_rolling_normil.hvplot(y='nltk_polarity')*ADA_rolling_normil.hvplot(y='nltk_polarity')*DOGE_rolling_normil.hvplot(y='nltk_polarity')

:Overlay
   .Curve.I   :Curve   [index]   (nltk_polarity)
   .Curve.II  :Curve   [index]   (nltk_polarity)
   .Curve.III :Curve   [index]   (nltk_polarity)
   .Curve.IV  :Curve   [index]   (nltk_polarity)

In [2401]:
Crypto_pos_sentiment=ETH.hvplot(y='POSITIVE', xlabel='Date', ylabel='Positive Sentiment', title='Positive Twitter Sentiment', label='ETH')*BTC.hvplot(y='POSITIVE', label='BTC')*ADA.hvplot(y='POSITIVE', label='ADA')*DOGE.hvplot(y='POSITIVE', label='DOGE')

In [2411]:
Crypto_neg_sentiment=ETH.hvplot(y='NEGATIVE', xlabel='Date', ylabel='Negative Sentiment', title='Negative Twitter Sentiment', label='ETH')*BTC.hvplot(y='NEGATIVE', label='BTC')*ADA.hvplot(y='NEGATIVE', label='ADA')*DOGE.hvplot(y='NEGATIVE', label='DOGE')

In [2441]:
BTC_sent=BTC_joined.hvplot(y='pct', label='BTC Sentiment', ylabel='Sentiment', xlabel='Date', title='Bitcoin and its Sentiment')*BTC_joined.hvplot(y='Close', label='BTC')*BTC_rolling_normil.hvplot(y='pct', label='BTC Rolling Sentiment')

In [2437]:
BTC_ROLL_sent=BTC_rolling_normil.hvplot(y='pct', label='BTC Sentiment', ylabel='Sentiment', title='Bitcoin and its Rolling Sentiment')*BTC_joined.hvplot(y='Close', label='BTC')

In [2367]:
BTC_rolling_normil.hvplot.bar(y='NEGATIVE', color='red')*BTC_rolling_normil.hvplot.bar(y='POSITIVE', color='Green')

:Overlay
   .Bars.I  :Bars   [index]   (NEGATIVE)
   .Bars.II :Bars   [index]   (POSITIVE)

In [2222]:
BTC_rolling_normil.hvplot(y='pct')*BTC_joined.hvplot(y='Close')

:Overlay
   .Curve.I  :Curve   [index]   (pct)
   .Curve.II :Curve   [index]   (Close)

In [2491]:
ETH_sent=ETH_joined.hvplot(y='pct', label='ETH Sentiment', ylabel='Sentiment', title='ETH and its Sentiment')*ETH_joined.hvplot(y='Close', label='ETH')*ETH_rolling_normil.hvplot(y='pct', label='ETH Rolling Sentiment')

In [2492]:
DOGE_sent=DOGE_joined.hvplot(y='pct', label='DOGE Sentiment', ylabel='Sentiment', title='DOGE and its Sentiment')*DOGE_joined.hvplot(y='Close', label='DOGE')*DOGE_rolling_normil.hvplot(y='pct', label='DOGE Rolling Sentiment')

In [2518]:
ADA_sent=ADA_joined.hvplot(y='pct', label='ADA Sentiment', ylabel='Sentiment', title='ADA and its Sentiment')*ADA_joined.hvplot(y='Close', label='ADA')*ADA_rolling_normil.hvplot(y='pct', label='ADA Rolling Sentiment')

In [2602]:
normil(BTC_joined.rolling(12).std()).hvplot(y='Close', label='BTC Moving Std')*BTC_rolling_normil.hvplot(y='pct', label='BTC Rolling Sentiment')

:Overlay
   .Curve.BTC_Moving_Std        :Curve   [index]   (Close)
   .Curve.BTC_Rolling_Sentiment :Curve   [index]   (pct)

In [2603]:
normil(ETH_joined.rolling(12).std()).hvplot(y='Close', label='ETH Moving Std')*ETH_rolling_normil.hvplot(y='pct', label='ETH Rolling Sentiment')

:Overlay
   .Curve.ETH_Moving_Std        :Curve   [index]   (Close)
   .Curve.ETH_Rolling_Sentiment :Curve   [index]   (pct)

In [2604]:
normil(ADA_joined.rolling(12).std()).hvplot(y='Close', label='ADA Moving Std')*ADA_rolling_normil.hvplot(y='pct', label='ADA Rolling Sentiment')

:Overlay
   .Curve.ADA_Moving_Std        :Curve   [index]   (Close)
   .Curve.ADA_Rolling_Sentiment :Curve   [index]   (pct)

In [2605]:
normil(DOGE_joined.rolling(12).std()).hvplot(y='Close', label='DOGE Moving Std')*DOGE_rolling_normil.hvplot(y='pct', label='DOGE Rolling Sentiment')

:Overlay
   .Curve.DOGE_Moving_Std        :Curve   [index]   (Close)
   .Curve.DOGE_Rolling_Sentiment :Curve   [index]   (pct)

In [2174]:
BTC_rolling_normil.hvplot(y='shifted_pct')*BTC_joined.hvplot(y='Close')

:Overlay
   .Curve.I  :Curve   [index]   (shifted_pct)
   .Curve.II :Curve   [index]   (Close)

In [2467]:
Corrl=[BTC_rolling_normil.corr()['pct']['Close'],ETH_rolling_normil.corr()['pct']['Close'],ADA_rolling_normil.corr()['pct']['Close'],DOGE_rolling_normil.corr()['pct']['Close']]

In [2468]:
Crypto=['BTC', 'ETH', 'ADA', 'DOGE']

In [2469]:
Corr=pd.DataFrame(index=Crypto, data=Corrl, columns=['Correlation'])

In [2470]:
Corr

,Correlation
BTC,0.408180
ETH,-0.493828
ADA,-0.316410
DOGE,-0.496203


In [2476]:
CORR_CRYPT=Corr.hvplot.bar(title='Correlation Values of Cryptos and Sentiment')

In [2219]:
BTC_rolling_normil_chg=BTC_rolling_normil.pct_change()

In [2220]:
BTC_rolling_normil_chg.corr()

,NEGATIVE,POSITIVE,pct,nltk_polarity,Open,High,Low,Close,Volume,Dividends,Stock Splits,shifted_positive,shifted_pct
NEGATIVE,1.000000,0.412341,-0.171221,0.246765,-0.028467,-0.002569,-0.039012,-0.020228,-0.018189,NaN,NaN,-0.173106,-0.049163
POSITIVE,0.412341,1.000000,0.345290,0.133030,-0.058127,-0.061576,0.034514,0.050893,0.016025,NaN,NaN,-0.161116,0.075415
pct,-0.171221,0.345290,1.000000,0.068511,-0.007283,-0.036036,0.094179,0.082234,-0.026684,NaN,NaN,0.026137,-0.022440
nltk_polarity,0.246765,0.133030,0.068511,1.000000,-0.072383,-0.027153,-0.008641,-0.005304,-0.070394,NaN,NaN,-0.018416,0.034074
Open,-0.028467,-0.058127,-0.007283,-0.072383,1.000000,0.814319,0.774529,0.565781,-0.008908,NaN,NaN,0.058675,-0.052732
High,-0.002569,-0.061576,-0.036036,-0.027153,0.814319,1.000000,0.939734,0.890948,0.060429,NaN,NaN,-0.007166,-0.062333
Low,-0.039012,0.034514,0.094179,-0.008641,0.774529,0.939734,1.000000,0.980877,-0.038038,NaN,NaN,-0.083124,-0.049920
Close,-0.020228,0.050893,0.082234,-0.005304,0.565781,0.890948,0.980877,1.000000,-0.019909,NaN,NaN,-0.100756,-0.059181
Volume,-0.018189,0.016025,-0.026684,-0.070394,-0.008908,0.060429,-0.038038,-0.019909,1.000000,NaN,NaN,0.082707,-0.035714
Dividends,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2757]:
intro = pn.Column(
    pn.pane.Markdown('## The Relationship between Twitter and The Market'), 
    pn.pane.Markdown('### Intro:'),
    pn.pane.Markdown('Our old mate Harold, phoned up the other day with some news! He had been fired from his job for stealing stationary and was now out on a limb, looking to start his own investment vehicle. Harold likes to spend alot of time on Twitter, he believes that Twitter could be powerful enough to influence markets. Harold has particular interest in twitters influence on cryptocurrencies and meme-stocks as he always hears on the news that social media is causing frenzies with these assets. He has asked us to investigate and see if there is truth to this'),
    pn.pane.Markdown('### Popularity of Crypto and Meme Stocks'),
    all_counts_plt*memestock_all_counts_plt,
    
)

In [2752]:
Cryptos = pn.Column(
    Crypto_Price,
    pn.Row(Crypto_pos_sentiment, Crypto_neg_sentiment),
    Crypto_Pct)

In [2753]:
corr_charts=pn.Column(
    pn.Row(BTC_sent,ETH_sent),
    pn.Row(DOGE_sent, ADA_sent),
    pn.Row(CORR_CRYPT),
    #Crypto_Pct
)

In [2754]:
meme_charts=pn.Column(
    pn.Row(MEMESTOCKS,MEMESTOCK_pos),
    pn.Row(MEMESTOCKSSENT),
    #pn.Row(CORR_CRYPT),
    #Crypto_Pct
)

In [2755]:
dashboard=pn.Tabs(
    ("Welcome", intro),
    ("Cryptos at a Glance", Cryptos),
    ("Crypto Correlation", corr_charts),
    ('MemeStonks',meme_charts),
    #('ADA',expensive_charts))
)

In [2756]:
dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Markdown(str)
        [3] Markdown(str)
        [4] HoloViews(Overlay)
    [1] Column
        [0] HoloViews(Overlay)
        [1] Row
            [0] HoloViews(Overlay)
            [1] HoloViews(Overlay)
        [2] HoloViews(Overlay)
    [2] Column
        [0] Row
            [0] HoloViews(Overlay)
            [1] HoloViews(Overlay)
        [1] Row
            [0] HoloViews(Overlay)
            [1] HoloViews(Overlay)
        [2] Row
            [0] HoloViews(Bars)
    [3] Column
        [0] Row
            [0] HoloViews(Overlay)
            [1] HoloViews(Overlay)
        [1] Row
            [0] HoloViews(Overlay)

:Scatter   [shifted_pct]   (Close)

In [2317]:
dashboard.save('dashboard.html', embed=True)

## MEMESTOCKS

In [1865]:
GME, GME_raw = complete_data('GME', 'gme OR gamestop OR gamestonk -is:retweet lang:en')
AMC, AMC_raw = complete_data('AMC', 'amc OR amc entertainment holdings -is:retweet lang:en')
TSLA, TSLA_raw = complete_data('TSLA', 'tsla OR tesla -is:retweet lang:en')

In [2237]:
GME.to_csv('GME.csv')
AMC.to_csv('AMC.csv')
TSLA.to_csv('TSLA.csv')

In [2537]:
GME_normil=normil(GME)
AMC_normil=normil(AMC)
TSLA_normil=normil(TSLA)

In [2657]:
gme = yf.Ticker("GME")
gme_stock = gme.history(
    start='2022-04-04',
    end='2022-04-11',
    interval='1D'
)
gme_price_normil=normil(gme_stock)

In [2658]:
amc = yf.Ticker("AMC")
amc_stock = amc.history(
    start='2022-04-04',
    end='2022-04-11',
    interval='1D'
)
amc_price_normil=normil(amc_stock)

In [2659]:
tsla = yf.Ticker("TSLA")
tsla_stock = tsla.history(
    start='2022-04-04',
    end='2022-04-11',
    interval='1D'
)
tsla_price_normil=normil(tsla_stock)

In [2660]:
GME_normil1=GME_normil.reset_index()
GME_normil1['time']=GME_normil1['time'].round('1D')
GME_normil1=GME_normil1.groupby('time').mean()
AMC_normil1=AMC_normil.reset_index()
AMC_normil1['time']=AMC_normil1['time'].round('1D')
AMC_normil1=AMC_normil1.groupby('time').mean()
TSLA_normil1=TSLA_normil.reset_index()
TSLA_normil1['time']=TSLA_normil1['time'].round('1D')
TSLA_normil1=TSLA_normil1.groupby('time').mean()

In [2661]:
GME.hvplot(x='time', y='nltk_polarity')*AMC.hvplot(x='time', y='nltk_polarity')*TSLA.hvplot(x='time', y='nltk_polarity')

:Overlay
   .Curve.I   :Curve   [time]   (nltk_polarity)
   .Curve.II  :Curve   [time]   (nltk_polarity)
   .Curve.III :Curve   [time]   (nltk_polarity)

In [2532]:
MEMESTOCK_pos=GME.hvplot(x='time', y='pct', ylabel='Percent of Tweets Positive', label='GME', title='Percentage of Tweets that are Positive')*AMC.hvplot(x='time', y='pct', label='AMC')*TSLA.hvplot(x='time', y='pct', label='TSLA')

In [2556]:
MEMESTOCKS=gme_price_normil.hvplot(y='Close',label='GME', title='Price of Popular Meme Stocks')*amc_price_normil.hvplot(y='Close', label='AMC')*tsla_price_normil.hvplot(y='Close', label='TSLA')

In [2674]:
MEMESTOCKSSENT=gme_price_normil.hvplot(y='Close', label='GME', ylabel='Sentiment', title='Stocks and Sentiments')*tsla_price_normil.hvplot(y='Close', label='TSLA')*amc_price_normil.hvplot(y='Close', label='AMC')*GME_normil1[:-2].hvplot(y='pct', label='GME Sentiment')*TSLA_normil1[:-2].hvplot(y='pct', label='TSLA Sentiment')*AMC_normil1[:-2].hvplot(y='pct', label='AMC Sentiment')

In [2671]:
tsla_price_normil.hvplot(y='Close', label='TSLA', ylabel='Sentiment', title='TSLA and its Sentiment')*TSLA_normil1[:-2].hvplot(y='pct', label='Sentiment')

:Overlay
   .Curve.TSLA      :Curve   [Date]   (Close)
   .Curve.Sentiment :Curve   [time]   (pct)

In [2623]:
tsla_price_normil.hvplot(y='Close', label='TSLA', ylabel='Sentiment', title='TSLA and its Sentiment')*TSLA_normil1[:-2].hvplot(y='pct', label='Sentiment')

:Overlay
   .Curve.I  :Curve   [Datetime]   (Close)
   .Curve.II :Curve   [time]   (pct)

In [2656]:
tsla_price_normil[:-1].hvplot(y='Close')*TSLA_normil[31:38].hvplot(y='pct')

:Overlay
   .Curve.I  :Curve   [index]   (Close)
   .Curve.II :Curve   [time]   (pct)

In [2224]:
GME_normil=GME_normil.reset_index()
GME_normil['time']=GME_normil['time'].round('1D')
GME_normil=GME_normil.groupby('time').mean()

In [2227]:
GME_normil

,NEGATIVE,POSITIVE,pct,nltk_polarity
time,,,,
2022-04-04 00:00:00+00:00,0.260552,0.712430,0.817614,0.496213
2022-04-05 00:00:00+00:00,0.487578,0.455634,0.525259,0.284666
2022-04-06 00:00:00+00:00,0.658929,0.227347,0.275696,0.486170
2022-04-07 00:00:00+00:00,0.570264,0.369122,0.428885,0.391469
2022-04-08 00:00:00+00:00,0.539643,0.385714,0.451520,0.257559
2022-04-09 00:00:00+00:00,0.409550,0.552351,0.625807,0.263596
2022-04-10 00:00:00+00:00,0.615327,0.328231,0.376906,0.476198


In [2564]:
GME_joined=pd.concat([GME_normil1,gme_price_normil], axis=1, join='inner')

In [2565]:
GME_joined

,NEGATIVE,POSITIVE,pct,nltk_polarity,Open,High,Low,Close,Volume,Dividends,Stock Splits


In [2566]:
GME_joined.corr()

,NEGATIVE,POSITIVE,pct,nltk_polarity,Open,High,Low,Close,Volume,Dividends,Stock Splits
NEGATIVE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
POSITIVE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pct,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
nltk_polarity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
High,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Low,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Close,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Volume,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Dividends,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2758]:
intro = pn.Column(
    pn.pane.Markdown('## The Relationship between Twitter and The Market'), 
    pn.pane.Markdown('### Intro:'),
    pn.pane.Markdown('Our old mate Harold, phoned up the other day with some news! He had been fired from his job for stealing stationary and was now out on a limb, looking to start his own investment vehicle. Harold likes to spend alot of time on Twitter, he believes that Twitter could be powerful enough to influence markets. Harold has particular interest in twitters influence on cryptocurrencies and meme-stocks as he always hears on the news that social media is causing frenzies with these assets. He has asked us to investigate and see if there is truth to this'),
    pn.pane.Markdown('### Popularity of Crypto and Meme Stocks'),
    all_counts_plt*memestock_all_counts_plt,
    
)

In [2759]:
Cryptos = pn.Column(
    Crypto_Price,
    pn.Row(Crypto_pos_sentiment, Crypto_neg_sentiment),
    Crypto_Pct)

In [2760]:
corr_charts=pn.Column(
    pn.Row(BTC_sent,ETH_sent),
    pn.Row(DOGE_sent, ADA_sent),
    pn.Row(CORR_CRYPT),
    #Crypto_Pct
)

In [2761]:
meme_charts=pn.Column(
    pn.Row(MEMESTOCKS,MEMESTOCK_pos),
    pn.Row(MEMESTOCKSSENT),
    #pn.Row(CORR_CRYPT),
    #Crypto_Pct
)

In [2762]:
dashboard=pn.Tabs(
    ("Welcome", intro),
    ("Cryptos at a Glance", Cryptos),
    ("Crypto Correlation", corr_charts),
    ('MemeStonks',meme_charts),
    #('ADA',expensive_charts))
)

In [2763]:
dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Markdown(str)
        [3] Markdown(str)
        [4] HoloViews(Overlay)
    [1] Column
        [0] HoloViews(Overlay)
        [1] Row
            [0] HoloViews(Overlay)
            [1] HoloViews(Overlay)
        [2] HoloViews(Overlay)
    [2] Column
        [0] Row
            [0] HoloViews(Overlay)
            [1] HoloViews(Overlay)
        [1] Row
            [0] HoloViews(Overlay)
            [1] HoloViews(Overlay)
        [2] Row
            [0] HoloViews(Bars)
    [3] Column
        [0] Row
            [0] HoloViews(Overlay)
            [1] HoloViews(Overlay)
        [1] Row
            [0] HoloViews(Overlay)

## COMMODITIES

In [1871]:
CORN, CORN_raw = complete_data('CORN', '#corn -is:retweet lang:en')

In [1879]:
WHEAT, WHEAT_raw = complete_data('WHEAT', 'wheat -is:retweet lang:en')

In [1880]:
OIL, OIL_raw = complete_data('OIL', 'crude oil OR oil market -is:retweet lang:en')

In [1881]:
VIX, VIX_raw = complete_data('VIX', 'VIX OR volatility index -is:retweet lang:en')

In [2249]:
corn = yf.Ticker("ZC=F")
corn_stock = corn.history(
    start='2022-04-04',
    end='2022-04-11',
    interval='1D'
)
corn_price_normil=normil(corn_stock)

In [2248]:
oil = yf.Ticker("CL=F")
oil_stock = oil.history(
    start='2022-04-04',
    end='2022-04-11',
    interval='1D'
)
oil_price_normil=normil(oil_stock)

In [2250]:
wheat = yf.Ticker("KE=F")
wheat_stock = wheat.history(
    start='2022-04-04',
    end='2022-04-11',
    interval='1D'
)
wheat_price_normil=normil(wheat_stock)

In [1883]:
CORN.to_csv('CORN.csv')
WHEAT.to_csv('WHEAT.csv')
OIL.to_csv('OIL.csv')
VIX.to_csv('VIX.csv')

In [2252]:
CORN_normil=normil(CORN)
WHEAT_normil=normil(WHEAT)
OIL_normil=normil(OIL)

In [2253]:
CORN_normil=CORN_normil.reset_index()
CORN_normil['time']=CORN_normil['time'].round('1D')
CORN_normil=CORN_normil.groupby('time').mean()
WHEAT_normil=WHEAT_normil.reset_index()
WHEAT_normil['time']=WHEAT_normil['time'].round('1D')
WHEAT_normil=WHEAT_normil.groupby('time').mean()
OIL_normil=OIL_normil.reset_index()
OIL_normil['time']=OIL_normil['time'].round('1D')
OIL_normil=OIL_normil.groupby('time').mean()

In [2269]:
CORN.hvplot(x='time', y='nltk_polarity')*WHEAT.hvplot(x='time', y='nltk_polarity')*OIL.hvplot(x='time', y='nltk_polarity')

:Overlay
   .Curve.I   :Curve   [time]   (nltk_polarity)
   .Curve.II  :Curve   [time]   (nltk_polarity)
   .Curve.III :Curve   [time]   (nltk_polarity)

In [2264]:
CORN.hvplot(x='time', y='pct')*WHEAT.hvplot(x='time', y='pct')*OIL.hvplot(x='time', y='pct')

:Overlay
   .Curve.I   :Curve   [time]   (pct)
   .Curve.II  :Curve   [time]   (pct)
   .Curve.III :Curve   [time]   (pct)

In [2259]:
corn_price_normil.hvplot(y='Close')*CORN_normil[:-2].hvplot(x='time', y='pct')

:Overlay
   .Curve.I  :Curve   [Date]   (Close)
   .Curve.II :Curve   [time]   (pct)

In [2260]:
oil_price_normil.hvplot(y='Close')*OIL_normil[:-2].hvplot(x='time', y='pct')

:Overlay
   .Curve.I  :Curve   [Date]   (Close)
   .Curve.II :Curve   [time]   (pct)

In [2261]:
wheat_price_normil.hvplot(y='Close')*WHEAT_normil[:-2].hvplot(x='time', y='pct')

:Overlay
   .Curve.I  :Curve   [Date]   (Close)
   .Curve.II :Curve   [time]   (pct)

## Australian

In [1897]:
CWN, CWN_raw = complete_data('CWN', 'cwn OR crown resorts OR crown casino -is:retweet lang:en')

In [1900]:
DMP, DMP_raw = complete_data('DMP', 'dominos pizza -is:retweet lang:en')

In [1903]:
HVN, HVN_raw = complete_data('HVN', 'harvey norman OR gerry harvey -is:retweet lang:en')

In [1906]:
PMV, PMV_raw = complete_data('PMV', 'premier investments -is:retweet lang:en')

In [1909]:
CWN.hvplot(x='time', y='nltk_polarity')*DMP.hvplot(x='time', y='nltk_polarity')*HVN.hvplot(x='time', y='nltk_polarity')*PMV.hvplot(x='time', y='nltk_polarity')

:Overlay
   .Curve.I   :Curve   [time]   (nltk_polarity)
   .Curve.II  :Curve   [time]   (nltk_polarity)
   .Curve.III :Curve   [time]   (nltk_polarity)
   .Curve.IV  :Curve   [time]   (nltk_polarity)

In [1910]:
CWN.hvplot(x='time', y='pct')*DMP.hvplot(x='time', y='pct')*HVN.hvplot(x='time', y='pct')*PMV.hvplot(x='time', y='pct')

:Overlay
   .Curve.I   :Curve   [time]   (pct)
   .Curve.II  :Curve   [time]   (pct)
   .Curve.III :Curve   [time]   (pct)
   .Curve.IV  :Curve   [time]   (pct)

In [ ]:
btc = yf.Ticker("BTC-USD")
btc_stock = btc.history(
    start='2022-04-04',
    end='2022-04-10',
    interval='60m'
)

In [ ]:
btc_chg=btc_stock.pct_change()

In [ ]:
btc_chg_normil=normil(btc_chg)

In [ ]:
btc_normil=normil(btc_stock)

In [1724]:
flairtest_normil['shifted_positive'] = flairtest_normil['POSITIVE'].shift(periods=-1)
flairtest_normil['shifted_negative'] = flairtest_normil['NEGATIVE'].shift(periods=-1)
flairtest_normil['shifted_pct'] = flairtest_normil['pct'].shift(periods=-1)

In [1756]:
joinednltk=pd.concat([nltktest_normil,btc_normil], axis=1, join='inner')

In [1757]:
joinednltk=joinednltk.drop(columns=['Open','High','Low','Dividends','Stock Splits'])

In [1758]:
joinednltk.corr()

,nltk_polarity,Close,Volume
nltk_polarity,1.000000,0.360770,-0.211527
Close,0.360770,1.000000,-0.231209
Volume,-0.211527,-0.231209,1.000000


In [ ]:
''

DASHBOARD